In [4]:
# --- 필수 라이브러리 임포트 ---
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
from ipywidgets import Layout, VBox, HBox
import time
import shutil
import io
import base64

# --- 폰트 설정 (Binder 환경을 위함, 메시지 OFF) ---
def setup_font_quietly():
    try:
        shutil.rmtree(matplotlib.get_cachedir(), ignore_errors=True)
    except:
        pass
    try:
        fm._load_fontmanager(try_read_cache=False)
        time.sleep(1)
    except:
        pass
    FONT_NAME_TO_SET = "NanumSquareRound"
    try:
        plt.rcParams['font.family'] = FONT_NAME_TO_SET
        plt.rcParams['axes.unicode_minus'] = False
        sns.set_style("whitegrid")
        plt.rc('font', family=FONT_NAME_TO_SET)
    except:
        pass

setup_font_quietly()

# --- 데이터 저장 클래스 ---
class KeywordDataManager:
    def __init__(self):
        self.keywords_data = pd.DataFrame(columns=[
            '키워드', 'DBpia결과수', '빅카인즈결과수', '교보문고결과수',
            '데이터가용성점수', '유레카지수', '덕질가능지수', '성장잠재력지수'
        ])
    def add_keyword(self, keyword, dbpia_count, bigkinds_count, kyobo_count,
                   data_score, eureka_score, fan_score, potential_score):
        new_data = pd.DataFrame([{
            '키워드': keyword,
            'DBpia결과수': dbpia_count,
            '빅카인즈결과수': bigkinds_count,
            '교보문고결과수': kyobo_count,
            '데이터가용성점수': data_score,
            '유레카지수': eureka_score,
            '덕질가능지수': fan_score,
            '성장잠재력지수': potential_score
        }])
        self.keywords_data = pd.concat([self.keywords_data, new_data], ignore_index=True)
        return self.keywords_data
    def calculate_data_availability_score(self, dbpia_count, bigkinds_count, kyobo_count):
        weighted_sum = (dbpia_count * 2) + bigkinds_count + kyobo_count
        if weighted_sum >= 500: return 4
        elif weighted_sum >= 200: return 3
        elif weighted_sum >= 50: return 2
        else: return 1

def get_color_palette(n):
    base_colors = ['#FF9AA2', '#FFB7B2', '#FFDAC1', '#E2F0CB', '#B5EAD7', '#C7CEEA', '#B5B9FF', '#ADE8F4']
    return (base_colors * (n // len(base_colors) + 1))[:n]

def display_header():
    display(HTML("""
    <div style="background: linear-gradient(90deg, #fccb90 0%, #d57eeb 100%);
                padding: 20px; border-radius: 15px; margin: 10px 0; text-align: center;">
        <h1 style="color: white; text-shadow: 2px 2px 4px #000000;">✨ 나만의 보석 키워드 발굴 시스템 ✨</h1>
        <h3 style="color: white;">당신의 특별한 연구 주제를 찾아서!</h3>
    </div>
    """))

def explanation_box(html_content):
    return HTML(f"""
    <div style="
        max-width:700px; margin:12px auto 18px auto;
        background: #f5f6fa; border-radius: 18px;
        border: 1.5px solid #e2e2e2;
        box-shadow:0 1.5px 8px #eee;
        padding:20px 18px 12px 18px;
        text-align:left; font-size:1.08em;
        line-height:1.7;
        color:#444;">
        {html_content}
    </div>
    """)

def display_animated_message(message, delay=0.02):
    out = widgets.Output()
    display(out)
    with out:
        display(HTML(f"""
        <div style="text-align:center; font-size:1.25em; color:#7e45ec; margin:10px 0;">
            {message}
        </div>
        """))
        time.sleep(delay * len(message) + 0.4)

def show_loading_animation():
    messages = [
        "🔍 키워드를 분석하고 있어요...",
        "✨ 보석을 발굴 중입니다...",
        "📊 데이터를 확인 중입니다...",
        "💫 마법의 알고리즘이 작동 중...",
        "🧠 키워드의 가능성을 계산 중..."
    ]
    for _ in range(2):
        for msg in messages:
            clear_output(wait=True)
            display(HTML(f"<h3 style='color:#6a0dad;text-align:center;'>{msg}</h3>"))
            time.sleep(0.4)
    clear_output(wait=True)
    display(HTML("<h3 style='color:#6a0dad;text-align:center;'>✅ 분석 완료!</h3>"))
    time.sleep(0.6)
    clear_output(wait=True)

def fig_to_base64(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    plt.close(fig)
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    return img_base64

def make_csv_download_widget(df, filename="키워드_분석_결과.csv"):
    buffer = io.StringIO()
    df.to_csv(buffer, index=False, encoding='utf-8-sig')
    payload = buffer.getvalue().encode('utf-8-sig')
    b64 = base64.b64encode(payload).decode()
    html = f'''
    <a download="{filename}" href="data:text/csv;base64,{b64}" target="_blank">
        <button style="background:#4267b2;color:white;padding:8px 18px;border:none;border-radius:6px;cursor:pointer;">
        📥 CSV 파일 다운로드
        </button>
    </a>
    '''
    return HTML(html)

def make_excel_download_widget(df, filename="키워드_분석_결과.xlsx"):
    buffer = io.BytesIO()
    with pd.ExcelWriter(buffer, engine='xlsxwriter') as writer:
        df.to_excel(writer, index=False, sheet_name="키워드분석")
    payload = buffer.getvalue()
    b64 = base64.b64encode(payload).decode()
    html = f'''
    <a download="{filename}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" target="_blank">
        <button style="background:#21a366;color:white;padding:8px 18px;border:none;border-radius:6px;cursor:pointer;">
        📊 엑셀 파일 다운로드
        </button>
    </a>
    '''
    return HTML(html)

# --- 메인 UI 및 기능 ---
class KeywordEvaluationApp:
    def __init__(self):
        self.data_manager = KeywordDataManager()
        self.setup_ui()

    def center_box(self, *widgets_list):
        return VBox([HBox([w], layout=Layout(justify_content='center')) for w in widgets_list], layout=Layout(align_items='center'))

    def setup_ui(self):
        display_header()
        display(HTML('<h2 style="text-align:center;">📝 새로운 키워드 입력</h2>'))
        display(HTML("<p style='text-align:center; font-size:0.85em; color:gray;'>*직접 검색 후 검색 결과 수기로 입력</p>"))
        self.keyword_input = widgets.Text(
            description='📌 키워드:',
            placeholder='연구하고 싶은 주제 키워드를 입력하세요',
            layout=Layout(width='80%'),
            style={'description_width': 'initial'}
        )
        self.dbpia_input = widgets.IntText(
            description='📚 DBpia 검색결과:', value=0, min=0, layout=Layout(width='60%'),
            style={'description_width': 'initial'}
        )
        self.bigkinds_input = widgets.IntText(
            description='📰 빅카인즈 검색결과:', value=0, min=0, layout=Layout(width='60%'),
            style={'description_width': 'initial'}
        )
        self.kyobo_input = widgets.IntText(
            description='📖 교보문고 검색결과:', value=0, min=0, layout=Layout(width='60%'),
            style={'description_width': 'initial'}
        )
        inputs_box = self.center_box(self.keyword_input, self.dbpia_input, self.bigkinds_input, self.kyobo_input)
        display(inputs_box)
        self.availability_output = widgets.Output()
        display(self.center_box(self.availability_output))
        self.calc_button = widgets.Button(
            description='🔍 데이터 가용성 점수 계산하기', button_style='info',
            layout=Layout(width='320px', min_width='200px', max_width='600px', height='44px'),
            style={'button_color': '#80C1FF', 'description_width': 'initial'}
        )
        self.calc_button.on_click(self.calculate_availability)
        display(self.center_box(self.calc_button))
        display(HTML('<h2 style="text-align:center;">💡 참신성/흥미도 점수 직접 선택하기</h2>'))
        display(explanation_box("""
        <b>✨ 유레카 지수란? (1~4점)</b><br>
        "이 주제, 다른 친구들은 잘 모르는 나만의 숨겨진 보석 같아!<br>새로운 관점으로 세상을 놀라게 할 수 있을 것 같아!"<br>
        <span style="font-size:0.97em;">4점: 완전히 새로운 발견! 아무도 연구하지 않은 분야<br>
        3점: 기존과는 다른 새로운 시각이 있는 주제<br>
        2점: 익숙하지만 나만의 독특한 관점이 있음<br>
        1점: 많은 사람들이 다루는 일반적인 주제</span>
        """))
        self.eureka_slider = widgets.IntSlider(
            value=2, min=1, max=4, step=1,
            description='✨ 유레카 지수:',
            layout=Layout(width='100%'),
            style={'description_width': 'initial'}
        )
        display(self.eureka_slider)
        display(explanation_box("""
        <b>💓 덕질 가능 지수란? (1~4점)</b><br>
        "이 주제만 생각하면 밤새도록 자료를 찾아보고 싶을 만큼 너무너무 재미있고 흥미진진해!<br>내 열정을 불태울 수 있어!"<br>
        <span style="font-size:0.97em;">4점: 완전 몰입! 이것만 생각하면 밤새 행복해짐<br>
        3점: 충분히 흥미롭고 파고들 가치가 있음<br>
        2점: 평범하게 재미있는 수준<br>
        1점: 별로 흥미롭지 않은 주제</span>
        """))
        self.fan_slider = widgets.IntSlider(
            value=2, min=1, max=4, step=1,
            description='💓 덕질 가능 지수:',
            layout=Layout(width='100%'),
            style={'description_width': 'initial'}
        )
        display(self.fan_slider)
        display(explanation_box("""
        <b>🚀 성장 잠재력 지수란? (1~4점)</b><br>
        "이 주제, 지금도 중요하지만 앞으로 우리 사회에 더 큰 영향을 줄 수 있는 엄청난 잠재력이 느껴져!<br>미래를 예측하고 대비하는 데 도움이 될 것 같아!"<br>
        <span style="font-size:0.97em;">        4점: 미래 사회를 바꿀 혁신적인 주제<br>
        3점: 앞으로 더 중요해질 가능성이 높은 주제<br>
        2점: 현재와 미래에 적당히 의미 있는 주제<br>
        1점: 미래 발전 가능성이 낮은 주제</span>
        """))
        self.potential_slider = widgets.IntSlider(
            value=2, min=1, max=4, step=1,
            description='🚀 성장 잠재력 지수:',
            layout=Layout(width='100%'),
            style={'description_width': 'initial'}
        )
        display(self.potential_slider)
        self.add_button = widgets.Button(
            description='✅ 키워드 추가하기', button_style='success',
            layout=Layout(width='320px', min_width='200px', max_width='600px', height='44px'),
            style={'description_width': 'initial'}
        )
        self.add_button.on_click(self.add_keyword_data)
        display(self.center_box(self.add_button))
        self.result_output = widgets.Output()
        display(self.center_box(self.result_output))

        self.graph_button = widgets.Button(
            description='📊 모든 키워드 그래프로 보기', button_style='warning',
            layout=Layout(width='340px', min_width='200px', max_width='600px', height='44px'),
            style={'description_width': 'initial'}
        )
        self.graph_button.on_click(self.show_graph)
        display(self.center_box(self.graph_button))
        self.graph_output = widgets.Output()
        display(self.center_box(self.graph_output))

    def calculate_availability(self, button):
        with self.availability_output:
            clear_output()
            if not self.keyword_input.value:
                display(HTML("<p style='color: red;text-align:center;'>⚠️ 키워드를 입력해주세요!</p>"))
                return
            show_loading_animation()
            dbpia = self.dbpia_input.value
            bigkinds = self.bigkinds_input.value
            kyobo = self.kyobo_input.value
            weighted_sum = (dbpia * 2) + bigkinds + kyobo
            score = self.data_manager.calculate_data_availability_score(dbpia, bigkinds, kyobo)
            emoji, message, color = {
                4: ("🎉", "풍년일세! 자료가 넘쳐나서 행복한 고민!", "#32CD32"),
                3: ("👌", "이 정도면 충분! 파고들 만해!", "#1E90FF"),
                2: ("💧", "가뭄의 단비... 자료가 좀 부족하지만, 희귀템을 노려볼까?", "#FFA500"),
                1: ("🏜️", "사막인가요... 정말 특별한 각오가 필요해!", "#FF4500"),
            }[score]
            display(HTML(f"""
            <div style="background-color: #f0f0f0; padding: 15px; border-radius: 10px; margin: 10px 0; text-align:center;">
                <h3 style="text-align:center;">"{self.keyword_input.value}" 키워드 분석 결과</h3>
                <p style="text-align:center;">📚 DBpia 검색결과: {dbpia}개 (가중치 2배 적용)</p>
                <p style="text-align:center;">📰 빅카인즈 검색결과: {bigkinds}개</p>
                <p style="text-align:center;">📖 교보문고 검색결과: {kyobo}개</p>
                <p style="text-align:center;">✨ 가중치 적용 총합: {weighted_sum}개</p>
                <h2 style="color: {color}; text-align:center;">데이터 가용성 점수: {score}점 {emoji}</h2>
                <h4 style="color: {color}; text-align:center;">{message}</h4>
            </div>
            """))

    def add_keyword_data(self, button):
        with self.result_output:
            clear_output()
            if not self.keyword_input.value:
                display(HTML("<p style='color: red;text-align:center;'>⚠️ 키워드를 입력해주세요!</p>"))
                return
            keyword = self.keyword_input.value
            dbpia = self.dbpia_input.value
            bigkinds = self.bigkinds_input.value
            kyobo = self.kyobo_input.value
            data_score = self.data_manager.calculate_data_availability_score(dbpia, bigkinds, kyobo)
            eureka_score = self.eureka_slider.value
            fan_score = self.fan_slider.value
            potential_score = self.potential_slider.value
            self.data_manager.add_keyword(
                keyword, dbpia, bigkinds, kyobo,
                data_score, eureka_score, fan_score, potential_score
            )
            display_animated_message(f"✨ '{keyword}' 키워드가 성공적으로 추가되었습니다! ✨")
            display(HTML('<h3 style="text-align:center;">📋 지금까지 추가된 키워드</h3>'))
            styled_df = self.data_manager.keywords_data.style \
                .background_gradient(cmap='YlGnBu', subset=['데이터가용성점수', '유레카지수', '덕질가능지수', '성장잠재력지수']) \
                .set_table_styles(
                    [
                        {'selector': 'th', 'props': [('text-align', 'center'), ('font-size', '1.05em'), ('padding', '12px 14px')]},
                        {'selector': 'td', 'props': [('text-align', 'center'), ('padding', '10px 14px')]}
                    ]
                ) \
                .set_properties(**{'text-align': 'center', 'padding': '10px 14px'})
            display(HTML('<div style="display: flex; justify-content: center;">' + styled_df.to_html() + '</div>'))
            self.keyword_input.value = ""
            self.dbpia_input.value = 0
            self.bigkinds_input.value = 0
            self.kyobo_input.value = 0
            self.eureka_slider.value = 2
            self.fan_slider.value = 2
            self.potential_slider.value = 2
            # 다운로드 버튼 새로 생성해서 display
            csv_btn_html = make_csv_download_widget(self.data_manager.keywords_data)._repr_html_()
            excel_btn_html = make_excel_download_widget(self.data_manager.keywords_data)._repr_html_()
            download_html = f'''
            <div style="display: flex; justify-content: center; gap: 18px; margin: 12px 0;">
                {csv_btn_html}
                {excel_btn_html}
            </div>
            '''
            display(HTML(download_html))


    def show_graph(self, button):
        with self.graph_output:
            clear_output()
            if len(self.data_manager.keywords_data) == 0:
                display(HTML("<p style='color: red;text-align:center;'>⚠️ 먼저 키워드를 추가해주세요!</p>"))
                return

            assessment_dropdown = widgets.Dropdown(
                options=[
                    ('유레카 지수 (참신성)', '유레카지수'),
                    ('덕질 가능 지수 (흥미도)', '덕질가능지수'),
                    ('성장 잠재력 지수 (미래성)', '성장잠재력지수'),
                    ('세 가지 지수의 평균', 'average')
                ],
                value='average', description='평가 기준:',
                layout=Layout(width='400px', min_width='250px', max_width='700px'),
                style={'description_width': 'initial'}
            )
            display(self.center_box(assessment_dropdown))
            output = widgets.Output()
            display(self.center_box(output))

            def update_graph(change):
                with output:
                    clear_output()
                    show_loading_animation()

                    sns.set_style("whitegrid")
                    plt.rc('font', family="NanumGothic")
                    plt.rcParams['axes.unicode_minus'] = False

                    assessment_type = assessment_dropdown.value
                    df = self.data_manager.keywords_data.copy()
                    if assessment_type == 'average':
                        df['종합점수'] = df[['유레카지수', '덕질가능지수', '성장잠재력지수']].mean(axis=1)
                        y_column = '종합점수'
                        title = '키워드 평가 맵: 데이터 가용성 vs 종합 점수'
                        y_label = '종합 점수 (참신성/흥미도/미래성)'
                    else:
                        y_column = assessment_type
                        title_mapping = {
                            '유레카지수': '키워드 평가 맵: 데이터 가용성 vs 유레카 지수 (참신성)',
                            '덕질가능지수': '키워드 평가 맵: 데이터 가용성 vs 덕질 가능 지수 (흥미도)',
                            '성장잠재력지수': '키워드 평가 맵: 데이터 가용성 vs 성장 잠재력 지수 (미래성)'
                        }
                        label_mapping = {
                            '유레카지수': '유레카 지수 (참신성)',
                            '덕질가능지수': '덕질 가능 지수 (흥미도)',
                            '성장잠재력지수': '성장 잠재력 지수 (미래성)'
                        }
                        title = title_mapping[assessment_type]
                        y_label = label_mapping[assessment_type]

                    fig, ax = plt.subplots(figsize=(12, 8))
                    colors = get_color_palette(len(df))
                    sns.scatterplot(
                        x='데이터가용성점수',
                        y=y_column,
                        data=df,
                        s=200,
                        hue='키워드',
                        palette=colors,
                        legend=False,
                        ax=ax
                    )
                    ax.axhline(y=2.5, color='gray', linestyle='--', alpha=0.5)
                    ax.axvline(x=2.5, color='gray', linestyle='--', alpha=0.5)
                    ax.fill_between([0, 2.5], 2.5, 5, alpha=0.1, color='gold')
                    ax.fill_between([2.5, 5], 2.5, 5, alpha=0.1, color='limegreen')
                    ax.fill_between([0, 2.5], 0, 2.5, alpha=0.1, color='tomato')
                    ax.fill_between([2.5, 5], 0, 2.5, alpha=0.1, color='dodgerblue')
                    ax.text(1.25, 3.75, "도전적인 보석\n(자료 부족, 높은 가치)", ha='center', fontsize=10)
                    ax.text(3.75, 3.75, "최고의 보석\n(자료 풍부, 높은 가치)", ha='center', fontsize=10)
                    ax.text(1.25, 1.25, "재고려 필요\n(자료 부족, 낮은 가치)", ha='center', fontsize=10)
                    ax.text(3.75, 1.25, "안정적 선택\n(자료 풍부, 낮은 가치)", ha='center', fontsize=10)
                    for i, row in df.iterrows():
                        ax.annotate(
                            row['키워드'],
                            (row['데이터가용성점수'], row[y_column]),
                            xytext=(5, 5),
                            textcoords='offset points',
                            fontsize=11, fontweight='bold'
                        )
                    ax.set_title(title, fontsize=16, pad=20)
                    ax.set_xlabel('데이터 가용성 점수', fontsize=14)
                    ax.set_ylabel(y_label, fontsize=14)
                    ax.set_xlim(0.5, 4.5)
                    ax.set_ylim(0.5, 4.5)
                    ax.set_xticks([1, 2, 3, 4])
                    ax.set_yticks([1, 2, 3, 4])
                    ax.grid(True, alpha=0.3)
                    fig.subplots_adjust(bottom=0.22)
                    img_data = fig_to_base64(fig)
                    display(HTML(
                        f'<div style="text-align:center;">'
                        f'<img src="data:image/png;base64,{img_data}" style="max-width:95%; height:auto; border-radius:18px; box-shadow:0 1.5px 8px #eee;">'
                        f'</div>'
                    ))
                    display(HTML('<div style="text-align:center;"><h3>✨ 보석 키워드 추천 ✨</h3></div>'))
                    if assessment_type == 'average':
                        df['average'] = df[['유레카지수', '덕질가능지수', '성장잠재력지수']].mean(axis=1)
                    quadrants = {
                        "최고의 보석 (자료 풍부, 높은 가치)": df[(df['데이터가용성점수'] >= 2.5) & (df[y_column] >= 2.5)],
                        "도전적인 보석 (자료 부족, 높은 가치)": df[(df['데이터가용성점수'] < 2.5) & (df[y_column] >= 2.5)],
                        "안정적 선택 (자료 풍부, 낮은 가치)": df[(df['데이터가용성점수'] >= 2.5) & (df[y_column] < 2.5)],
                        "재고려 필요 (자료 부족, 낮은 가치)": df[(df['데이터가용성점수'] < 2.5) & (df[y_column] < 2.5)]
                    }
                    for category, keywords in quadrants.items():
                        if len(keywords) > 0:
                            display(HTML(f'<div style="text-align:center;"><h4>{category}</h4>'))
                            for i, row in keywords.iterrows():
                                badge_color = {
                                    "최고의 보석 (자료 풍부, 높은 가치)": "#28a745",
                                    "도전적인 보석 (자료 부족, 높은 가치)": "#ffc107",
                                    "안정적 선택 (자료 풍부, 낮은 가치)": "#17a2b8",
                                    "재고려 필요 (자료 부족, 낮은 가치)": "#dc3545",
                                }[category]
                                emoji = {
                                    "최고의 보석 (자료 풍부, 높은 가치)": "★",
                                    "도전적인 보석 (자료 부족, 높은 가치)": "☆",
                                    "안정적 선택 (자료 풍부, 낮은 가치)": "○",
                                    "재고려 필요 (자료 부족, 낮은 가치)": "△",
                                }[category]
                                display(HTML(f"""
                                <div style="margin: 5px 0; padding: 10px; border-radius: 5px; background-color: #f8f9fa; border-left: 5px solid {badge_color};text-align:center;">
                                    <span style="font-weight: bold;">{emoji} {row['키워드']}</span>
                                    <span style="float: right; padding: 2px 8px; border-radius: 10px; background-color: {badge_color}; color: white;">점수: {row[y_column]:.1f}</span>
                                </div>
                                """))
                            display(HTML('</div>'))

            assessment_dropdown.observe(update_graph, names='value')
            update_graph(None)

# --- 앱 실행 ---
app = KeywordEvaluationApp()

Could not save font_manager cache [Errno 2] No such file or directory: '/home/jovyan/.cache/matplotlib/fontlist-v390.json.matplotlib-lock'


IntSlider(value=2, description='✨ 유레카 지수:', layout=Layout(width='100%'), max=4, min=1, style=SliderStyle(descr…

IntSlider(value=2, description='💓 덕질 가능 지수:', layout=Layout(width='100%'), max=4, min=1, style=SliderStyle(des…

IntSlider(value=2, description='🚀 성장 잠재력 지수:', layout=Layout(width='100%'), max=4, min=1, style=SliderStyle(de…